### Notebook to run the complete eval of our method for Handmade questions

https://docs.google.com/spreadsheets/d/1xgkQt_1NiwXEZhZUvTypgecG5wMCl8_brc6w1bhaOtE/edit?gid=0#gid=0



## Step 0 : download videos

In [20]:
import pandas as pd

df_questions = pd.read_csv('Handmade_questions - Feuille 1.csv')\

df_questions = df_questions.dropna(subset=['Question']) #drop any rows that aren't filled yet

In [2]:
df_questions

,VideoID,Question,Answer1 (corr),Answer2,Answer3,Answer4,youtubeURL,Unnamed: 7,Unnamed: 8,https://www.youtube.com/watch?v=,Unnamed: 10
0,zwbCD3CUFIs,What color is the spatula on the cutting board?,Green,Red,Blue,Black,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN,NaN
1,zwbCD3CUFIs,What green ingredient is added to the pan ?,Spring onions,Green pepper,Green beans,Cucumber,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN,NaN
2,zwbCD3CUFIs,What is the purpose of the large blue object o...,To provide heat to cook,It is a trash can,It is a pan,It's a decoration,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN,NaN
3,zwbCD3CUFIs,How many prawns are cooking in the pan?,approximately 25,About 4 big prawns,approximately 80 prawns,there are no prawns,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN,NaN
5,XzgR14o6b80,What is the color if the liquid the two steaks...,Yellow-brown,Red,Green,Transparent,https://www.youtube.com/watch?v=XzgR14o6b80,NaN,NaN,NaN,NaN
6,XzgR14o6b80,What color is the hair of the man cooking?,Blond,Black,Medium Brown,White,https://www.youtube.com/watch?v=XzgR14o6b80,NaN,NaN,NaN,NaN
7,XzgR14o6b80,Which country are the cabbages from?,They are chinese,They are french,They are british,They are mexican,https://www.youtube.com/watch?v=XzgR14o6b80,NaN,NaN,NaN,(transcript says chinese cabbage)
8,XzgR14o6b80,How much wasabi is on the spoon ?,About a thumbnail size,NaN,A full spoon,A single drop,https://www.youtube.com/watch?v=XzgR14o6b80,NaN,NaN,NaN,NaN
10,vow-kxTPatc,How many cups of flour are required ?,2 cups,1 cup,5 cups,NaN,https://www.youtube.com/watch?v=vow-kxTPatc,NaN,NaN,NaN,1:02 in the vid
11,vow-kxTPatc,What color is the top of the woman ?,Red - brown,Yellow,White,Black,https://www.youtube.com/watch?v=vow-kxTPatc,NaN,NaN,NaN,NaN


In [3]:
ALL_URLS = df_questions['VideoID'].unique().tolist()

In [11]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

base_path = '/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage'

for URL in ALL_URLS:

    os.makedirs(f"{base_path}/{URL}", exist_ok=True)
    url = f"https://www.youtube.com/watch?v={URL}"
    
    yt = YouTube(url, on_progress_callback = on_progress)
    print(yt.title)
    ys = yt.streams.get_highest_resolution()
    ys.download(f"{base_path}/{URL}", filename='video')

    caption = yt.captions.get_by_language_code('a.en')

    #save captions to ./
    caption.save_captions(f"{base_path}/{URL}/caption.srt")

Singapore-Style Curry Fried Rice Noodles | Asian Bites


/scratch/1029067/ipykernel_559228/2911685545.py:17: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Experience Culinary Bliss with Gordon Ramsay Beef Tacos with Wasabi Mayo
Naan Bread Recipe | How To Make Naan Bread at Home by Manjula

/scratch/1029067/ipykernel_559228/2911685545.py:17: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


/scratch/1029067/ipykernel_559228/2911685545.py:17: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Easy New England Clam Chowder Recipe | The Frugal Chef


/scratch/1029067/ipykernel_559228/2911685545.py:17: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


## Step 1 : Generate the CSV from input videos

In [12]:
# args are
# ('--v', type=str, help='path to the video file (mp4)')
# ('--o', type=str, help='path to output folder')
# ('--s', type=str, help='path to SRT files (transcript)')
# ('--i', type=int, help='frame interval')


for URL in ALL_URLS:
    video_path = f"{base_path}/{URL}/video.mp4"
    srt_path = f"{base_path}/{URL}/caption.srt"

    #launch script /home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/step1_video_to_frames_csv.py with python

    os.system(f"python /home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/step1_video_to_frames_csv.py --v {video_path} --o {base_path}/{URL} --s {srt_path} --i 60 > /dev/null")

## Step 2 : Generate FAISS embeddings from the CSVs

In [4]:

import json
import os
import numpy as np
from numpy.linalg import norm
import cv2
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from termcolor import cprint
import faiss
import pandas as pd

base_path = '/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage'

processor = BridgeTowerProcessor.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir="/home/hice1/bpopper3/scratch/hug_face_files/hub_cache")
model = BridgeTowerForContrastiveLearning.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir="/home/hice1/bpopper3/scratch/hug_face_files/hub_cache")


def encode_single_image_text(image_path, text):
    print(image_path)
    print(text)
    image = Image.open(image_path)
    encoding = processor(image, text, return_tensors="pt")
    outputs = model(**encoding)
    return outputs['cross_embeds'].detach().numpy()





for URL in ALL_URLS:
    input_folder = f"{base_path}/{URL}"
    
    
        #load ./paired_image_data.csv
    df_mapping = pd.read_csv(f'{input_folder}/image_transcript_mapping.csv')
    #rows :             index,image_path,transcript_text,timestamp

    #create empty FAISS index
    

    # Create embeddings for each row
    embeddings = np.zeros((len(df_mapping), 512))
    for _, row in tqdm(df_mapping.iterrows()):


        embedding = encode_single_image_text(row['image_path'], row['transcript_text'])
        #reshape to (512,)
        embedding = embedding.reshape(512)
        embeddings[row['index'],:] = embedding

        print(embedding.shape)

    # Initialize FAISS index
    dimension = 512  # dimension of your embeddings
    index = faiss.IndexFlatL2(dimension)

    # Add embeddings to the FAISS index
    index.add(embeddings)

    # Create a mapping between FAISS ids and dataframe indices
    id_to_index = {i: idx for i, idx in enumerate(df_mapping.index)}

    # Save the FAISS index
    faiss.write_index(index, f"{input_folder}/faiss_database.bin")
    print('saved faiss index')

/home/hice1/bpopper3/.conda/envs/vlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hice1/bpopper3/.conda/envs/vlm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BridgeTowerForContrastiveLearning were not initialized from the model checkpoint at BridgeTower/bridgetower-large-itm-mlm-itc and are newly initialized: ['logit_scale']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
0it [00:00, ?it/

/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage/zwbCD3CUFIs/frames/frame_0000.jpg
hey guys I'm Amy and I'm Julie we are the Duncan sisters welcome to the scoffs kitchen today in our Asian bite series we are doing a Singapore fried rice noodle with curry is you can see it's 


1it [00:07,  7.09s/it]

(512,)
/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage/zwbCD3CUFIs/frames/frame_0060.jpg
hey guys I'm Amy and I'm Julie we are the Duncan sisters welcome to the scoffs kitchen today in our Asian bite series we are doing a Singapore fried rice noodle with curry is you can see it's this bright yellow sort of concoction and it's super yummy so let's get 


2it [00:09,  4.28s/it]

...

(512,)
/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage/UUlfKJ9M_pQ/frames/frame_7440.jpg
a lot thank you I will have this recipe on the website ready for you guys to come and print it in the meantime eat well without going broke I'm going to see you guys soon 


125it [04:47,  2.30s/it]

(512,)
/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage/UUlfKJ9M_pQ/frames/frame_7500.jpg
on the website ready for you guys to come and print it in the meantime eat well without going broke I'm going to see you guys soon 


126it [04:49,  2.30s/it]

(512,)
saved faiss index


## Step 3 : Chat with FAISS Db as RAG

In [5]:
import faiss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from PIL import Image
from termcolor import cprint
import pandas as pd
os.environ["HUGGINGFACE_HUB_CACHE"] = "/home/hice1/bpopper3/scratch/hug_face_files/hub_cache"
os.environ["HF_HOME"] = "/home/hice1/bpopper3/scratch/hug_face_files/hf_home"

#by default dont load if not needed
processor = None
model = None
is_loaded = False
def load_bridgetower():
    global index
    global df
    global is_loaded
    if is_loaded == False:
        print('loading BRIDGETOWER ##############################################################')
        is_loaded = True
        global processor
        global model
        processor = BridgeTowerProcessor.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=os.getenv("HUGGINGFACE_HUB_CACHE"))
        model = BridgeTowerForContrastiveLearning.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=os.getenv("HUGGINGFACE_HUB_CACHE"))


def encode_text_query(text):
    global index
    global df
    if not is_loaded:
        load_bridgetower()
    #create random Image 32x32
    image = Image.fromarray(np.random.randint(0,255,(32,32,3),dtype=np.uint8))
    encoding = processor(image, text, return_tensors="pt")
    outputs = model(**encoding)
    #reshape to (512,)

    return outputs['text_embeds'].detach().numpy().reshape(-1)


def get_n_closest_index(embedding,n):
    global index
    global df
    D, I = index.search(np.array([embedding]), n)
    #I[0] has the indexes of the n closest images
    return I[0]

def set_index(folder_path):
    global index
    global df
    index = faiss.read_index(folder_path+'/faiss_database.bin')
    df = pd.read_csv(folder_path+'/image_transcript_mapping.csv')



In [6]:
def return_image_and_enhanced_query(question,answer_a,answer_b,answer_c,answer_d,folder_path):
    global index
    global df
    set_index(folder_path)
    text_embedding = encode_text_query(question)
    
    #return query with 1 closest image/text pair to enhance the query

    closest_index = get_n_closest_index(text_embedding,1)
    print(f"Closest index is {closest_index[0]} for query '{question}'")
    return create_enhanced_conversation(df.iloc[closest_index[0]]['transcript_text'],question,Image.open(df.iloc[closest_index[0]]['image_path']),answer_a,answer_b,answer_c,answer_d)


def create_enhanced_conversation(textt,question,image,answer_a,answer_b,answer_c,answer_d):
    global index
    global df
    
    return [{
        "role": "user",
        "content": [
            {"type": "text", "text" : f"""Please answer this question : 'What is 2+2'
            This transcript can also help you answer : 'Addition is a mathematical commutative operation that' .
            It is possible that the image helps you answer the question, but it is possible that it does not
            Answer A: 4
            Answer B: 6
            Answer C: 9
            Answer D: 12"""}
        ],
    },
    {
        "role": "assistant",
        "content": [
   
            {"type": "text", "text" : "A"}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text" : f"""Please answer this question : '{question}'
            This transcript can also help you answer : '{textt}' .
            It is possible that the image helps you answer the question, but it is possible that it does not
            Answer A: {answer_a}
            Answer B: {answer_b}
            Answer C: {answer_c}
            Answer D: {answer_d}"""}
        ],
    }],image

In [7]:
#This script loads LLaVA so it's on a big GPU
import json
import os
import numpy as np
from numpy.linalg import norm
import cv2
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from termcolor import cprint

os.environ["HUGGINGFACE_HUB_CACHE"] = "/home/hice1/bpopper3/scratch/hug_face_files/hub_cache"
os.environ["HF_HOME"] = "/home/hice1/bpopper3/scratch/hug_face_files/hf_home"



########################## LOADING LLAVA ##########################


print("beginning to load LLaVA")
#load on cuda:0

llava_processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", device="cuda:0",cache_dir=os.getenv("HUGGINGFACE_HUB_CACHE"))


from transformers import LlavaNextForConditionalGeneration, BitsAndBytesConfig

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

llava_model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", quantization_config=quantization_config, device_map="auto",cache_dir=os.getenv("HUGGINGFACE_HUB_CACHE"))


beginning to load LLaVA


Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
Loading checkpoint shards: 100%|██████████| 4/4 [00:29<00:00,  7.31s/it]


In [18]:
print('test')

test


In [13]:
print(df_questions.keys())

Index(['VideoID', 'Question', 'Answer1 (corr)', 'Answer2', 'Answer3',
       'Answer4', 'youtubeURL', 'Unnamed: 7', 'Unnamed: 8',
       'https://www.youtube.com/watch?v=', 'Unnamed: 10'],
      dtype='object')


In [19]:
df_questions

,index,image_path,transcript_text,timestamp
0,0,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,[Music],0.0000
1,1,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,[Music],2.5025
2,2,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,[Music] first thing we're going to do is uh cook,5.0050
3,3,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,[Music] first thing we're going to do is uh co...,7.5075
4,4,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,first thing we're going to do is uh cook some ...,10.0100
...,...,...,...,...
121,121,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,friends and you sub and you comment thank you ...,302.8025
122,122,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,thank you thank you thank you I really really ...,305.3050
123,123,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,how much I appre appreciate your support a lot...,307.8075
124,124,/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pip...,a lot thank you I will have this recipe on the...,310.3100


In [26]:
#iterate over all the questions
import torch.nn.functional as F

total_correct = 0

for index, row in df_questions.iterrows():
    URL = row['VideoID']
    question = row['Question']
    correct_answer = row['Answer1 (corr)']
    wrong_answer1 = row['Answer2']
    wrong_answer2 = row['Answer3']
    wrong_answer3 = row['Answer4']

    #shuffle and assign answer_a,b,c,d
    answers = [correct_answer,wrong_answer1,wrong_answer2,wrong_answer3]
    np.random.shuffle(answers)

    answer_a = answers[0]
    answer_b = answers[1]
    answer_c = answers[2]
    answer_d = answers[3]
    print(f"A : {answer_a}")
    print(f"B : {answer_b}")
    print(f"C : {answer_c}")
    print(f"D : {answer_d}")

    folder_path = f"{base_path}/{URL}"

    
    conv,image = return_image_and_enhanced_query(question,answer_a,answer_b,answer_c,answer_d,folder_path)

    prompt = llava_processor.apply_chat_template(conv, add_generation_prompt=True)
    inputs = llava_processor(image,prompt, return_tensors="pt").to("cuda:0")
    print(f"Question: {question}")
    with torch.no_grad():
        # Get the model's output logits directly
        model_outputs = llava_model(**inputs)
        logits = model_outputs.logits[0, -1, :]
        
        # Apply softmax to get probabilities
        probabilities = F.softmax(logits, dim=0)
        
        answer_tokens = {
            'C': 334,
            'A': 330,
            'B': 365,
            'D': 384
        }
        
        # Check probabilities of known answer tokens
        answer_probs = {label: probabilities[token_id].item() 
                        for label, token_id in answer_tokens.items()}
        
        # Find the top answer among the known tokens
        top_answer = max(answer_probs, key=answer_probs.get)
        
        def top_answer_to_idx(character):
            print(character)
            if character == "A":
                return 0
            if character == "B":
                return 1
            if character == "C":
                return 2
            if character == "D":
                return 3
        
        top_answer_text = answers[top_answer_to_idx(top_answer)]
        print(f"Top Answer: {top_answer} / {top_answer_text}")
        print(f"correct answer : {correct_answer}")
        
        if top_answer_text == correct_answer:
            print("CORRECT")
            total_correct+=1
        else:
            print("INCORRECT")
            
        print(f"Probabilities of known answers:")
        for label, prob in answer_probs.items():
            print(f"{label}: {prob:.4f}")
            
            
        
        # If the top token is not one of the known answers
        top_token_id = torch.argmax(probabilities).item()
        if top_token_id not in answer_tokens.values():
            top_token = llava_processor.tokenizer.convert_ids_to_tokens([top_token_id])[0]
            print(f"\nTop token not in known answers:")
            print(f"Token: {top_token}")
            print(f"Token ID: {top_token_id}")
            print(f"Probability: {probabilities[top_token_id].item():.4f}")

    print('----------------------------------------------------------------------')




    


A : Blue
B : Black
C : Red
D : Green
Closest index is 65 for query 'What color is the spatula on the cutting board?'
Question: What color is the spatula on the cutting board?
B
Top Answer: B / Black
correct answer : Green
INCORRECT
Probabilities of known answers:
C: 0.0048
A: 0.0519
B: 0.9202
D: 0.0026
----------------------------------------------------------------------
A : Green beans
B : Green pepper
C : Cucumber
D : Spring onions
Closest index is 17 for query 'What green ingredient is added to the pan ?'
Question: What green ingredient is added to the pan ?
B
Top Answer: B / Green pepper
correct answer : Spring onions
INCORRECT
Probabilities of known answers:
C: 0.0735
A: 0.2754
B: 0.4007
D: 0.1835
----------------------------------------------------------------------
A : It is a trash can
B : It's a decoration
C : It is a pan
D : To provide heat to cook
Closest index is 87 for query 'What is the purpose of the large blue object on the table ?'
Question: What is the purpose of the

In [27]:
print(total_correct/len(df_questions))

0.625


In [ ]:
# question,answer_a,answer_b,answer_c,answer_d,folder_path = 'what color is the omelet?','4','5','green','yellow','/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage/zwbCD3CUFIs'


# conv,image = return_image_and_enhanced_query(question,answer_a,answer_b,answer_c,answer_d,'/home/hice1/bpopper3/scratch/LLaVA_RAG/RAG_Pipeline/HANDMADE_QUESTIONS_PIPELINE/storage/zwbCD3CUFIs')

# prompt = llava_processor.apply_chat_template(conv, add_generation_prompt=True)
# inputs = llava_processor(image,prompt, return_tensors="pt").to("cuda:0")

# import torch.nn.functional as F

# with torch.no_grad():
#     # Get the model's output logits directly
#     model_outputs = llava_model(**inputs)
#     logits = model_outputs.logits[0, -1, :]
    
#     # Apply softmax to get probabilities
#     probabilities = F.softmax(logits, dim=0)
    
#     answer_tokens = {
#         'C': 334,
#         'A': 330,
#         'B': 365,
#         'D': 384
#     }
    
#     # Check probabilities of known answer tokens
#     answer_probs = {label: probabilities[token_id].item() 
#                     for label, token_id in answer_tokens.items()}
    
#     # Find the top answer among the known tokens
#     top_answer = max(answer_probs, key=answer_probs.get)
    
#     print(f"Top Answer: {top_answer}")
#     print(f"Probabilities of known answers:")
#     for label, prob in answer_probs.items():
#         print(f"{label}: {prob:.4f}")
    
#     # If the top token is not one of the known answers
#     top_token_id = torch.argmax(probabilities).item()
#     if top_token_id not in answer_tokens.values():
#         top_token = llava_processor.tokenizer.convert_ids_to_tokens([top_token_id])[0]
#         print(f"\nTop token not in known answers:")
#         print(f"Token: {top_token}")
#         print(f"Token ID: {top_token_id}")
#         print(f"Probability: {probabilities[top_token_id].item():.4f}")
    
# #     # Get the top 5 most likely tokens
# #     top_5_values, top_5_indices = torch.topk(probabilities, 5)
# #     print(top_5_indices)
    
    
# #     # 334 is C
# #     # 330 is A
# #     # 365 is B
# #     # 384 is D
# #     # Decode the token indices back to tokens
# #     top_5_tokens = llava_processor.tokenizer.convert_ids_to_tokens(top_5_indices.tolist())
    
# #     # Print the top 5 tokens with their probabilities
# #     print("Top 5 Most Likely Next Tokens:")
# #     for token, prob in zip(top_5_tokens, top_5_values):
# #         print(f"Token: {token}, Probability: {prob.item():.4f}")
